In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, StructType, StructField


In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.3.2")
                .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") 
                .config("spark.hadoop.fs.s3a.access.key", "minio") 
                .config("spark.hadoop.fs.s3a.secret.key", "minio123") 
                .config("spark.hadoop.fs.s3a.path.style.access", "true") 
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
                .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
                .config("spark.sql.catalog.spark_catalog.type", "hadoop")
                .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://commerce/warehouse/") 
                .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") 
                .appName("feature customer")
         .getOrCreate())

25/02/13 21:38:25 WARN Utils: Your hostname, Gauravs-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
25/02/13 21:38:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/saurabh/Desktop/workspace/realtime_fraud_detection_system/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/saurabh/.ivy2/cache
The jars for the packages stored in: /Users/saurabh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c914ee51-01da-4a19-bc47-fb4cec6d1de6;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in ce

In [3]:
def execute_query(query: str):
    spark.sql(query).show(truncate=False)

In [4]:

df = spark.read \
    .format("iceberg") \
    .load("s3a://commerce/warehouse/payment/transactions/")
df.createOrReplaceTempView('transactions')

25/02/13 21:39:45 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
df.printSchema()

root
 |-- tx_id: integer (nullable = true)
 |-- tx_datetime: timestamp (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- terminal_id: integer (nullable = true)
 |-- tx_amount: decimal(10,2) (nullable = true)
 |-- row_created_timestamp: timestamp (nullable = true)
 |-- row_updated_timestamp: timestamp (nullable = true)



In [6]:
execute_query("""Select * from transactions""")

+-------+-------------------+-----------+-----------+---------+-------------------------+-------------------------+
|tx_id  |tx_datetime        |customer_id|terminal_id|tx_amount|row_created_timestamp    |row_updated_timestamp    |
+-------+-------------------+-----------+-----------+---------+-------------------------+-------------------------+
|2051528|2025-01-01 02:07:39|2405       |7044       |41.65    |2025-02-13 21:31:08.09608|2025-02-13 21:31:08.09608|
|2051661|2025-01-01 02:49:43|2249       |3336       |15.23    |2025-02-13 21:31:08.09608|2025-02-13 21:31:08.09608|
|2052048|2025-01-01 04:33:58|4535       |401        |67.42    |2025-02-13 21:31:08.09608|2025-02-13 21:31:08.09608|
|2051507|2025-01-01 01:54:30|4693       |1159       |97.03    |2025-02-13 21:31:08.09608|2025-02-13 21:31:08.09608|
|2051862|2025-01-01 03:44:19|736        |8019       |39.72    |2025-02-13 21:31:08.09608|2025-02-13 21:31:08.09608|
|2051919|2025-01-01 04:01:07|2674       |8291       |6.39     |2025-02-1

In [ ]:
# # 100 data (txn_Date_time) -> t - 1 customer_id 

# execute_query("""
# select  customer_id,current_date as dt,
#         count(*) as customer_id_nb_tx_1day_window,
#       avg(tx_amount) as customer_id_avg_amount_1day_window
#         from transactions  where date(tx_datetime) = current_date -1
#         group by customer_id,tx_datetime
# """)

In [ ]:

# execute_query("""
# select  customer_id,current_date as dt,
#         count(*) as customer_id_nb_tx_7day_window,
#         avg(tx_amount) as customer_id_avg_amount_7day_window
#         from transactions  where date(tx_datetime) between current_date -7 and current_date
#         group by customer_id,tx_datetime
# """)

In [ ]:

# execute_query("""
# select  customer_id,current_date as dt,
#         count(*) as customer_id_nb_tx_30day_window,
#         avg(tx_amount) as customer_id_avg_amount_30day_window
#         from transactions  where date(tx_datetime) between current_date -30 and current_date
#         group by customer_id,tx_datetime
# """)

In [7]:
execute_query("""create or replace temp view latest_data as 
WITH one_day_window AS (
    SELECT  
        customer_id,
        
        COUNT(*) AS customer_id_nb_tx_1day_window,
        AVG(tx_amount) AS customer_id_avg_amount_1day_window
    FROM transactions
    WHERE date(tx_datetime) = current_date - 1
    GROUP BY customer_id
),
seven_day_window AS (
    SELECT  
        customer_id,
        COUNT(*) AS customer_id_nb_tx_7day_window,
        AVG(tx_amount) AS customer_id_avg_amount_7day_window
    FROM transactions
    WHERE date(tx_datetime) BETWEEN current_date - 7 AND current_date
    GROUP BY customer_id
),
thirty_day_window AS (
    SELECT  
        customer_id,
        COUNT(*) AS customer_id_nb_tx_30day_window,
        AVG(tx_amount) AS customer_id_avg_amount_30day_window
    FROM transactions
    WHERE date(tx_datetime) BETWEEN current_date - 30 AND current_date
    GROUP BY customer_id
)
SELECT 
    one_day.customer_id,
    current_date AS dt,
    one_day.customer_id_nb_tx_1day_window,
    one_day.customer_id_avg_amount_1day_window,
    seven_day.customer_id_nb_tx_7day_window,
    seven_day.customer_id_avg_amount_7day_window,
    thirty_day.customer_id_nb_tx_30day_window,
    thirty_day.customer_id_avg_amount_30day_window
FROM one_day_window AS one_day
LEFT JOIN seven_day_window AS seven_day
    ON one_day.customer_id = seven_day.customer_id 
LEFT JOIN thirty_day_window AS thirty_day
    ON one_day.customer_id = thirty_day.customer_id
""")


++
||
++
++



In [8]:
execute_query(""" select * from latest_data""")

+-----------+---+-----------------------------+----------------------------------+-----------------------------+----------------------------------+------------------------------+-----------------------------------+
|customer_id|dt |customer_id_nb_tx_1day_window|customer_id_avg_amount_1day_window|customer_id_nb_tx_7day_window|customer_id_avg_amount_7day_window|customer_id_nb_tx_30day_window|customer_id_avg_amount_30day_window|
+-----------+---+-----------------------------+----------------------------------+-----------------------------+----------------------------------+------------------------------+-----------------------------------+
+-----------+---+-----------------------------+----------------------------------+-----------------------------+----------------------------------+------------------------------+-----------------------------------+



In [9]:
df.show

<bound method DataFrame.show of DataFrame[tx_id: int, tx_datetime: timestamp, customer_id: int, terminal_id: int, tx_amount: decimal(10,2), row_created_timestamp: timestamp, row_updated_timestamp: timestamp]>

In [10]:
execute_query("""
CREATE TABLE IF NOT EXISTS payment.feature_customer (
    customer_id INT,
    customer_id_nb_tx_1day_window INT,
    customer_id_avg_amount_1day_window FLOAT,
    customer_id_nb_tx_7day_window INT,
    customer_id_avg_amount_7day_window FLOAT,
    customer_id_nb_tx_30day_window INT,
    customer_id_avg_amount_30day_window FLOAT,
    x_location FLOAT,
    y_location FLOAT,
    row_created_timestamp TIMESTAMP,
    row_updated_timestamp TIMESTAMP
)
USING iceberg
""").show(truncate=False)



++
||
++
++



AttributeError: 'NoneType' object has no attribute 'show'